In [1]:
import requests
from io import BytesIO
from pprint import pprint
from collections import Counter
import numpy as np
from datetime import datetime
from bs4 import BeautifulSoup
from collections import defaultdict 
from pytz import timezone
import json
import nltk
import random
import re
import os.path
import urllib.parse
from urllib.parse import quote
import urllib.request
from IPython import display
from azure.cognitiveservices.search.visualsearch import VisualSearchClient
from azure.cognitiveservices.search.visualsearch.models import (
    VisualSearchRequest,
    CropArea,
    ImageInfo,
    Filters,
    KnowledgeRequest,
)
from msrest.authentication import CognitiveServicesCredentials


Azure Pricing: https://azure.microsoft.com/en-us/pricing/details/cognitive-services/search-api/

In [2]:
# Image Insights API
subscription_key = "0a71d4ad9ae84586bf712a4204288dbe" # S9 pricing tier,  COST: $3 per 1,000 transactions
visual_url = "https://api.bing.microsoft.com/v7.0/images/visualsearch"
headers = {"Ocp-Apim-Subscription-Key" : subscription_key}

In [3]:
# Image Search API
imgSearch_subscription_key = "aab34a07c6724b078e275a8e2dece7f2" # S3 pricing tier，COST: $4 per 1,000 transactions
search_url = "https://api.bing.microsoft.com/v7.0/images/search"
imgSearch_headers = {"Ocp-Apim-Subscription-Key" : imgSearch_subscription_key}

In [4]:
def contentUrl_to_displayUrl(contentUrl):
    prefix = 'https://commons.wikimedia.org/wiki/'
    tokens = contentUrl.split('/')
    if 'px-' in tokens[-1]:
        return prefix + 'File:'+tokens[-2]
    else:
        return prefix + 'File:'+tokens[-1]
def get_description_by_url(desurl):
    try:
        with urllib.request.urlopen(desurl) as f:
            html = f.read().decode('utf-8')
    except KeyboardInterrupt:
        raise
    except:
        #print(desurl)
        return ""
    soup = BeautifulSoup(html, 'html.parser')
    result = soup.find_all('div',class_= "description mw-content-ltr en", limit=1)
    des = " "
    for r in result:
        soup_short = BeautifulSoup(str(r), 'html.parser')
        for row in soup_short.prettify().split('\n'):
            if not "<" in row:
                des += row.strip()
        des += '\n'
    return des

def get_filename_and_hash_by_url(desurl):
    filename = urllib.parse.unquote(desurl).split("/")[-1].replace("File:", "").replace(".jpg", "")
    _hash = hash(filename)
    return (filename, _hash)

In [5]:
def update_imgUrl(imgUrl):
    if not "/thumb/" in imgUrl:
        filename = imgUrl.split("/")[-1]
        sep = imgUrl.find("wikipedia/commons/") + len("wikipedia/commons/")
        return imgUrl[:sep] + "thumb/" + imgUrl[sep:] + "/800px-" + filename
    else:
        filename = imgUrl.split("/")[-2]
        sep = imgUrl.find(filename) + len(filename)
        last = imgUrl.split("/")[-1]
        n = last[:last.find("px-")]
        try:
            if int(n) <= 800:
                return imgUrl
            else: 
                return imgUrl[:sep] + "/800px-" + filename
        except:
            return imgUrl[:sep] + "/800px-" + filename
        

In [6]:
def displayUrl_to_imgUrl(displayUrl, drop = False):
    try:
        with urllib.request.urlopen(displayUrl) as f:
            html = f.read().decode('utf-8')
    except KeyboardInterrupt:
        raise
    except:
        #print("can't open url: ", displayUrl)
        return "CANNOT_FOUND"

    soup = BeautifulSoup(html, 'html.parser')
    result = soup.find_all('a',class_= "mw-thumbnail-link", limit=10)
    idx = 0
    default = ""
    for r in result:
        soup_short = BeautifulSoup(str(r), 'html.parser')
        for a in soup_short.find_all('a', href=True):
            href = a['href']
            if idx == 0: 
                default = a['href']
                idx += 1
            if '800px' in href:
                return href
    if len(default)>0: return default
    else:
        if drop:
            print("DROP")
            return "CANNOT_FOUND"
        else:
            result = soup.find_all('div',class_= "fullImageLink", limit=1)
            for r in result:
                soup_short = BeautifulSoup(str(r), 'html.parser')
                for im in soup_short.find_all('img'):
                    if max(int(im['width']), int(im['height']))>=600 and min(int(im['width']), int(im['height']))>=400:
                        return im['src']
                    return "SIZE"

In [7]:
def img_filter(contentUrl):
    url_blocklist = ['seal', 'sign ', 'pdf', 'gif', 'icon', 'notice', 'cartoon', 'publish', 'menu', 'logo', 'svg', 'webm', 'cover', 'page', \
                     'ogg', 'album', 'flickr', 'poster', 'ogv', 'banner', 'tif', 'montage']
    
    
    # Restrict to Wikimedia images
    if not contentUrl.startswith('https://upload.wikimedia.org/wikipedia/'): return False, "", "", "", "wrong prefix"
    
    displayUrl = contentUrl_to_displayUrl(contentUrl)
    description = get_description_by_url(displayUrl).replace("English:", "").strip('\n')
    concat_url_des = " "+displayUrl+description+" "
    
    # Avoid images with bad keywords
    if any(b in concat_url_des for b in url_blocklist): return False, "", "", "", "keyword"
    
    # Avoid images with "18xx, 17xx, 16xx", which probably indicates the image is old
    #regex = re.compile('.*\D(18|17|16)\d{2}\D.*')
    #if not re.match(regex, concat_url_des)==None: return False, "", "", "", "year"
    imgUrl = displayUrl_to_imgUrl(displayUrl, drop=False)
    if imgUrl == 'CANNOT_FOUND' or imgUrl=='SIZE': return False, "", "", "", imgUrl
    
    imgUrl = update_imgUrl(imgUrl) 
    # This is kind of tedious since both "displayUrl_to_imgUrl" and "update_imgUrl" are updating the image src
    # I used "displayUrl_to_imgUrl()" at first and find that it can return image sizes but does not process src urls correctly. 
    # So I added "update_imgUrl()" later
    return True, displayUrl, description, imgUrl, ""

In [8]:
def img_to_querySuggestion(imgUrl, PRINT=False):
    data_dict = {
       "imageInfo" : {
            "url" : "{}".format(imgUrl),
            
        },
        "knowledgeRequest": {
            "filters" : {
              "site" : "wikimedia.org"
            }
        }
    }
    data_json = json.dumps(data_dict, indent = 4)
    files = {'knowledgeRequest': ('imageInfo', data_json)}
    try:
        response = requests.post(visual_url, headers=headers, files=files)
        # COST: $3 per 1,000 transactions
        if PRINT: print_json(response.json())
        response.raise_for_status()
        
        ## Print related queries
        expansion = {'RelatedSearches': [], 'DocumentLevelSuggestions': [], 'BestRepresentativeQuery': []}
        for actn in response.json()['tags'][0]['actions']:
            if 'RelatedSearches' in actn['actionType']:
                for rs in actn['data']['value']:
                    expansion['RelatedSearches'].append(rs['displayText'])
            if 'DocumentLevelSuggestions' in actn['actionType']:
                for rs in actn['data']['value']:
                    expansion['DocumentLevelSuggestions'].append(rs['displayText'])
            if 'BestRepresentativeQuery' in actn['actionType']:
                expansion['BestRepresentativeQuery'].append(actn['displayName'])
        
    except KeyboardInterrupt:
        raise
    except:
        return {'RelatedSearches': [], 'DocumentLevelSuggestions': [], 'BestRepresentativeQuery': []}
    return expansion

In [9]:
'''
Args:
    search_term (str)
Return:
    new_queries (list)
'''
def expand_query(search_term, PRINT=False):
    if PRINT: print(search_term)
    
    params  = {"q": search_term, "license": "ModifyCommercially", "imageType": "photo"}
    try:
        response = requests.get(search_url, headers=imgSearch_headers, params=params)
        # COST: $4 per 1,000 transactions
    except:
        print("Error when search using: ", search_term)
        raise
    while response.status_code == 429 or response.status_code == 403:
        time.sleep(2)
        response = requests.get(search_url, headers=headers, params=params)
    response.raise_for_status()
    search_results = response.json()
    
    new_queries = set([])
    
    # Avoid queries with bad keywords
    q_blocklist = ['flickr', 'logo', 'wikimedia', "quote", "clip art", "map", "card", "illustration", "drawing", "worksheet", "symbol",\
                   "icon", "paper", "tv", "cloth", "cartoon", "gif", "album", "sign"]
        
    if 'queryExpansions' in list(search_results.keys()):
        for qe in search_results['queryExpansions']:
            if any(b in qe['text'].lower() for b in q_blocklist): continue
            if not qe['text'] in new_queries:
                new_queries.add(qe['text'])
        
    if 'pivotSuggestions' in list(search_results.keys()):
        for pivot in search_results['pivotSuggestions']:
            for sug in pivot['suggestions']:
                if any(b in sug['text'].lower() for b in q_blocklist): continue
                if not sug['text'] in new_queries:
                    new_queries.add(sug['text'])
    if 'relatedSearches' in list(search_results.keys()):
        for rs in search_results["relatedSearches"]:
            if any(b in rs['text'].lower() for b in q_blocklist): continue
            if not rs['text'] in new_queries:
                new_queries.add(rs['text'])
    
    if PRINT: print(len(new_queries))
    new_queries - set('search_term')
    return new_queries

In [10]:
'''
Args:
    imgUrl: src url of the anchor image
    count: number of visually similar images to return
    last: The returned list of the API call arrange images by visual similarity. 
        However, we don't want the distrator to be too similar to the gold img. So a bunch of top ones should be skipped.
        E.g. count = 10, last = 20, this function will return visually similar images with rank 11-20.

Return:
    similar_images.keys(): a list of src urls
    similar_images: a dict where keys are src urls and values are wikimedia page urls
    
'''
def search_visually_similar_late(imgUrl, count, last = 20, PRINT = False):
    data_dict = {
       "imageInfo" : {
            "url" : "{}".format(imgUrl),
            
        },
        "knowledgeRequest": {
            "filters" : {
              "site" : "wikimedia.org"
            }
        }
    }
    data_json = json.dumps(data_dict, indent = 4)
    #pprint(data_json)
    files = {'knowledgeRequest': ('imageInfo', data_json)}
    similar_images = {}
    try:
        response = requests.post(visual_url, headers=headers, files=files)
        # COST: $3 per 1,000 transactions
        if PRINT: print_json(response.json())
        response.raise_for_status()
        
        for actn in response.json()['tags'][0]['actions']:
            if 'VisualSearch' in actn['actionType']:
                length = len(actn['data']['value'])
                if length < count-1:
                    #print(length, imgUrl)
                    for i in range(length):
                        similar_images[actn['data']['value'][i]['contentUrl']] = actn['data']['value'][i]['hostPageUrl']
                elif length<last:
                    for i in range(length-count, length):
                        similar_images[actn['data']['value'][i]['contentUrl']] = actn['data']['value'][i]['hostPageUrl']
                else:
                    for i in range(last-count, last):
                        similar_images[actn['data']['value'][i]['contentUrl']] = actn['data']['value'][i]['hostPageUrl']
    except KeyboardInterrupt:
        raise
    except:
        print("Error in when searching visually similar images: ", imgUrl)
        return [], {}
    return list(similar_images.keys()), similar_images

In [20]:
'''
Args:
    query (str): search term
    num_results_api: number of images returned by API 
        (should be greater than 'max_search_result_per_query' since some images will be filtered out)
Return:
    d (dict): key is img filename, values are imgUrl, Wikimedia page url, and description
    bad_img_statistics (Counter): Store the reason why some bad images are dropped. 
        E.g. not from Wikimedia; img size too small; bad keywords; etc
    bad_query_statistics (Counter): Store the reason why a search term is bad
        I used bad_query_statistics to flag bad topics that won't yield interesting images when I was mining image prompts for MTurk
'''

def seed_to_img(query, max_search_result_per_query, min_search_result_per_query, num_results_api=35, site = "", print_error_detail=False):
    d = {}
    params  = {"q": query+site, "license": "ModifyCommercially", "imageType": "photo", "count": num_results_api}

    response = requests.get(search_url, headers=imgSearch_headers, params=params)
    # COST: $4 per 1,000 transactions
    while response.status_code == 429 or response.status_code == 403:
        print("sleep 2 seconds")
        time.sleep(2)
        response = requests.get(search_url, headers=imgSearch_headers, params=params)
    response.raise_for_status()
    search_results = response.json()
    
    url_blocklist = ['seal', 'sign', 'pdf', 'gif', 'icon', 'notice', 'cartoon', 'publish', 'menu', 'logo', 'svg', 'webm', 'cover', 'page', \
                     'ogg', 'album', 'flickr', 'poster', 'ogv', 'banner', 'tif']
    
    search_result = 0
    conse_fail = 0
    fail = 0
    bad_img_statistics = Counter()
    bad_query_statistics = Counter()
    for img in search_results["value"]:
        
        contentUrl = img['contentUrl']
        page = img['hostPageUrl']
        #print("returned by api: ",contentUrl)
        if contentUrl.startswith('http://'):
            contentUrl = contentUrl[:4]+"s"+contentUrl[4:]
        
        # Filter out bad images, get filename, description, update imgUrl
        valid, displayUrl, description, imgUrl, drop_reason= img_filter(contentUrl)
        
        if not valid: 
            if print_error_detail: print(drop_reason+" "+page)
            bad_img_statistics[drop_reason] += 1
            continue
        conse_fail = 0
        key, _hash = get_filename_and_hash_by_url(displayUrl)
        d[key] = {}
        d[key]['displayUrl'] = displayUrl
        d[key]['imgUrl'] = imgUrl
        # I tried to use hash(filename) or timestamp as img unique identifier but it turned out to be a bad idea
        #d[key]['_hash'] = _hash
        #d[key]['timestamp'] = datetime.now(tz=timezone('US/Eastern')).strftime("%y%m%d%H%M%S%f")
        d[key]['des'] = description
        
        if len(d) >= max_search_result_per_query: break
    
    if len(d) < min_search_result_per_query: 
        bad_query_statistics['too_few_survivors'] += 1
        return {}, bad_img_statistics, bad_query_statistics
    #print("Finish search for ", query)
    return d, bad_img_statistics, bad_query_statistics  
    

### Search Term --> Imgs (via image search API)
https://docs.microsoft.com/en-us/bing/search-apis/bing-image-search/quickstarts/rest/python

In [22]:
d, _, __ = seed_to_img("flint and steel", 10, 5, print_error_detail=True)
print(d.keys())
print(d)
print()
#pprint(list(d.items())[0])

wrong prefix http://wiki.minetest.net/Flint_and_Steel
wrong prefix http://flickr.com/photos/schwuk/3195851605
wrong prefix http://wiki.minetest.net/Flint_and_Steel
wrong prefix http://skyquazatheairrider.deviantart.com/art/Terraria-1-2-honey-fountain-3-405074790
CANNOT_FOUND https://en.wikipedia.org/wiki/Tinderbox
wrong prefix https://www.flickr.com/photos/22616984@N07/7785234602/
wrong prefix http://mineblocks.com/1/wiki/index?title=File:Flint.png
wrong prefix https://mineblocks.com/1/wiki/index.php?title=Flint
wrong prefix https://www.vitantica.net/2018/01/16/acciarino-antico-moderno/
wrong prefix http://wiki.minetest.net/Flint
wrong prefix https://klexikon.zum.de/wiki/Datei:Flint_wall,_West_Winterslow_-_geograph.org.uk_-_889238.jpg
wrong prefix https://www.flickr.com/photos/martin7d2/29851186244
wrong prefix http://j1nzo.deviantart.com/art/Charizard-Sprite-205307739
wrong prefix http://gaming.stackexchange.com/questions/29789/through-what-ways-can-tnt-be-ignited
wrong prefix https:/

### Search Term --> More related queries (via image search API)
https://docs.microsoft.com/en-us/bing/search-apis/bing-image-search/quickstarts/rest/python

In [15]:
expand_query("Seattle Space Needle")

{'Best Seattle Skyline Space Needle',
 'Big Ben',
 'CN Tower',
 'Diagram of Seattle Space Needle',
 'Empire State Building',
 'Gateway Arch',
 'Golden Gate Bridge',
 'Hoover Dam',
 'Liberty Bell',
 'Mount Rainier',
 'Mount Rushmore National Memorial',
 'One World Trade Center',
 'Seattle Center',
 'Seattle Great Wheel',
 'Seattle Seahawks Space Needle',
 'Seattle Space Needle Art',
 'Seattle Space Needle Construction',
 'Seattle Space Needle Fire',
 'Seattle Space Needle History',
 'Seattle Space Needle Night',
 'Seattle Space Needle Outline',
 'Seattle Space Needle Paintings',
 'Seattle Space Needle Restaurant Menu',
 'Seattle Space Needle Silhouette',
 'Seattle Space Needle Skyline',
 'Seattle Space Needle Top',
 'Seattle Space Needle Tour',
 'Seattle Tower Space Needle',
 'Space Needle Building',
 'Space Needle Dimensions',
 'Space Needle Fireworks',
 'Space Needle Height',
 'Space Needle Observation Deck',
 'Space Needle Seattle United States',
 'Space Needle Seattle WA',
 'Space N

### Img --> Related searches (via image insights API)
https://docs.microsoft.com/en-us/azure/cognitive-services/bing-image-search/quickstarts/python

Not every image have RelatedSearches. More than 60% of the imgs actually get empty fields from this API call.

In [16]:
img_to_querySuggestion("https://upload.wikimedia.org/wikipedia/commons/thumb/f/f0/CMU_from_36th_floor.jpg/834px-CMU_from_36th_floor.jpg")

{'BestRepresentativeQuery': [],
 'DocumentLevelSuggestions': [],
 'RelatedSearches': ['Carnegie Mellon Campus',
  'Carnegie Mellon Campus Map',
  'Carnegie Mellon University Map',
  'Carnegie Mellon University Dorms',
  'Carnegie Mellon Logo',
  'Carnegie Mellon Robotics',
  'Carnegie Mellon University Computer Science',
  'Carnegie Mellon Students',
  'Carnegie Mellon University Qatar',
  'Carnegie Mellon Buildings',
  'Carnegie Mellon University Mascot',
  'Carnegie Mellon University Engineering',
  'Carnegie Mellon University Tepper',
  'Carnegie Mellon University Location',
  'Carnegie Mellon Seal',
  'Carnegie Mellon University School of Computer Science',
  'CMU Pittsburgh',
  'Carnegie Mellon University Library',
  'Carnegie Mellon University Australia',
  'Pittsburgh PA Carnegie Mellon',
  'Robotics Institute Carnegie Mellon University',
  'Tepper School of Business',
  'Carnegie Mellon University Symbol',
  'Carnegie Mellon College',
  'Carnegie Mellon Wallpaper',
  'Pennsylva

### Img --> Visually Similar Images (via image insights API)
https://docs.microsoft.com/en-us/azure/cognitive-services/bing-image-search/quickstarts/python


Apply **img_filter()** to the imgUrls of visually similar images if necessary.

In [17]:
imgUrl = "https://upload.wikimedia.org/wikipedia/commons/thumb/f/f0/CMU_from_36th_floor.jpg/834px-CMU_from_36th_floor.jpg"
similar_urls, similar_images = search_visually_similar_late(imgUrl, 10, 20)
print(type(similar_images))
print(similar_images[similar_urls[0]])

<class 'dict'>
http://commons.wikimedia.org/wiki/file:kelvingrove_art_gallery_and_museum_from_the_university_of_glasgow_tower.jpg


### Draft

In [25]:
question = "Which occurred first: the Debra Dean character appearing on East Enders or the airing of the first episode from the 22nd season of Home and Away?"
fact = "As the series progresses, it is revealed that Diva has five chevaliers to aid her: Amshel Goldsmith, Solomon Goldsmith, Karl Fei-Ong, James Ironside, and Nathan Mahler. They have formed their own organization, Cinq Fl\u00e8ches, which they use to create additional chiropterans from Diva's blood."

In [27]:
pos_tags = ['NN', 'NNP', 'NNS', 'NNPS']# 'VB', 'VBD', 'VBG', 'VBP', 'VBZ', 'JJ','JJR', 'JJS', 'RB', 'RBR', 'RBS']

In [28]:
s = " ".join([x[0] for x in nltk.pos_tag(fact.split()) if x[1] in pos_tags])
print(s)
s = " ".join([x[0] for x in nltk.pos_tag(question.split()) if x[1] in pos_tags])
print(s)

series progresses, Diva chevaliers her: Amshel Goldsmith, Solomon Goldsmith, Karl Fei-Ong, James Ironside, Nathan Mahler. organization, Cinq Flèches, chiropterans Diva's blood.
Debra Dean character East Enders airing episode season Home Away?


In [43]:
seed_to_img("Amshel Goldsmith", 10, 5)

returned by api:  https://upload.wikimedia.org/wikipedia/it/thumb/5/54/Amshel_Goldsmith.jpg/260px-Amshel_Goldsmith.jpg
can't open url:  https://commons.wikimedia.org/wiki/File:Amshel_Goldsmith.jpg
returned by api:  https://upload.wikimedia.org/wikipedia/it/5/54/Amshel_Goldsmith.jpg
can't open url:  https://commons.wikimedia.org/wiki/File:Amshel_Goldsmith.jpg
returned by api:  https://static.miraheze.org/allthetropeswiki/3/3b/Amshel01_1253.png
returned by api:  https://upload.wikimedia.org/wikipedia/en/thumb/e/e1/All_Diva's_Chevaliers.png/220px-All_Diva's_Chevaliers.png
can't open url:  https://commons.wikimedia.org/wiki/File:All_Diva's_Chevaliers.png
returned by api:  https://upload.wikimedia.org/wikipedia/en/thumb/e/e1/All_Diva's_Chevaliers.png/330px-All_Diva's_Chevaliers.png
can't open url:  https://commons.wikimedia.org/wiki/File:All_Diva's_Chevaliers.png
returned by api:  https://upload.wikimedia.org/wikipedia/it/thumb/f/f0/David_Blood.jpg/220px-David_Blood.jpg
can't open url:  htt

({},
 Counter({'CANNOT_FOUND': 9, 'SIZE': 1, 'wrong prefix': 2}),
 Counter({'too_few_survivors': 1}))